In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from collections import Counter
import pandas as pd
import openpyxl
import random
import string 
import nltk

In [3]:
df = pd.read_excel('final_data_file.xlsx')

In [4]:
nltk.download('punkt')

# Cümleleri kelimelerine ayır ve yeni bir DataFrame oluştur
word_list = []
for sentence in df['sentence']:  # 'Cümle Sütunu' yazan yere, gerçek sütun adınızı yazmalısınız
    words = word_tokenize(str(sentence))
    word_list.extend(words)
    
word_freq = Counter(word_list)    

# En çok tekrar eden 20 kelimeyi bul
top_20_words = word_freq.most_common(20)

# Sonuçları yazdır
print("En çok tekrar eden 20 kelime ve frekansları:")
for word, count in top_20_words:
    print(f"{word}: {count} kere")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aysei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


En çok tekrar eden 20 kelime ve frekansları:
psikoloji: 10305 kere
bir: 2578 kere
var: 555 kere
ben: 552 kere
ters: 522 kere
benim: 484 kere
bi: 475 kere
mi: 460 kere
kadar: 447 kere
olarak: 419 kere
iyi: 419 kere
yok: 419 kere
değil: 409 kere
sosyoloji: 334 kere
sonra: 332 kere
psikolojik: 304 kere
sosyal: 296 kere
artık: 287 kere
kalmadı: 258 kere
olan: 258 kere


In [5]:

# Word2Vec modelini oluşturun
model = Word2Vec(sentences=[word_list], vector_size=100, window=5, min_count=1, workers=4)

# En çok tekrar eden 20 kelimenin her biri için en benzer 5 kelimeyi bulun
similar_words_dict = {}
for word, _ in top_20_words:
    if word in model.wv:
        similar_words = model.wv.most_similar(word, topn=5)
        similar_words_dict[word] = [similar_word for similar_word, _ in similar_words]

# Sonuçları yazdır
for word, similar_words in similar_words_dict.items():
    print(f"\n{word} kelimesi için benzer kelimeler:")
    for similar_word in similar_words:
        print(similar_word)


psikoloji kelimesi için benzer kelimeler:
yönetemediğin
bir
eğitimlerimizi
boşanmapsikoloji
yıl

bir kelimesi için benzer kelimeler:
inanır
etrafından
yetkinlikse
psikoloji
işlevsiz

var kelimesi için benzer kelimeler:
pdr
aygün
yansıtacağını
farketmeksizin
dokunur

ben kelimesi için benzer kelimeler:
recmel
olumlamalar
uğurlu
yılların
ektiği

ters kelimesi için benzer kelimeler:
denemiştim
galatasarayı
halifetin
birinci
aydınserdarkuru

benim kelimesi için benzer kelimeler:
doçenti
eşittir
sessizliği
havaalanında
yönetildiğinde

bi kelimesi için benzer kelimeler:
olur
çocuklarıma
psikoloji
alıntılanma
okuluna

mi kelimesi için benzer kelimeler:
price
gözlük
kitapönerisi
danışanıma
jwkxkf

kadar kelimesi için benzer kelimeler:
uçtu
katmalı
vücudda
birisi
profesörlükile

olarak kelimesi için benzer kelimeler:
zorlanmışlar
carlosum
sınava
bağırsakta
afişe

iyi kelimesi için benzer kelimeler:
düzelten
yanlislikla
bilişsel
hakkinda
oyunsun

yok kelimesi için benzer kelimeler:
zirvesine
ki

In [6]:


# Excel dosyasını yükleyin (dosya adını ve sayfa adını değiştirmeyi unutmayın)
df = pd.read_excel('final_data_file.xlsx')  # 'Sayfa1' yerine kendi sayfa adınızı kullanın

# 'sentence' sütunundaki NaN değerleri temizleyin
df = df.dropna(subset=['sentence'])

# Cümleleri ayırın ve Word2Vec modelini oluşturun
word_list = [word_tokenize(str(sentence)) for sentence in df['sentence']]
model = Word2Vec(sentences=word_list, vector_size=100, window=5, min_count=1, workers=4)

# TfidfVectorizer kullanarak cümle vektörlerini oluşturun
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['sentence'])

# Rastgele 5 cümle seçin
#random.seed(42)  # Rastgele seçimi her çalıştırmada aynı sonuçları almak için seed kullanıyoruz
selected_indices = random.sample(range(len(df)), 5)

for idx in selected_indices:
    selected_sentence = df.loc[idx, 'sentence']
    print(f"\nSeçilen Cümle ({idx+1}. satır): {selected_sentence}")

    # Seçilen cümleyi Word2Vec modeline göre vektörle
    selected_vector = model.wv[word_tokenize(str(selected_sentence))]

    # Cosine Similarity kullanarak en benzer 3 cümleyi bulun
    similarities = cosine_similarity(tfidf_matrix, tfidf_vectorizer.transform([selected_sentence]))
    similar_indices = similarities.flatten().argsort()[:-5:-1]  # En büyük üç benzerlik indeksi

    # Benzer cümleleri yazdırın
    print("En Benzer 3 Cümle:")
    for similar_idx in similar_indices[1:]:  # İlk indeks seçilen cümle olacak, onu atla
        similar_sentence = df.loc[similar_idx, 'sentence']
        print(f"{similar_idx+2}. satır: {similar_sentence}")



Seçilen Cümle (2374. satır): felsefe grubuna sosyolojipsikoloji hayırlı olsun mucadelefelsefe
En Benzer 3 Cümle:
6877. satır: hayırlı olsun psikoloji si bozuk olan diğer partililer
4863. satır: psikoloji felsefe
10762. satır: felsefe psikoloji

Seçilen Cümle (9856. satır): cidden psikoloji alanı kıymetli benim kadar geç olursa olsun elbet bir gün okuyacağımı biliyorum
En Benzer 3 Cümle:
57. satır: benim psikoloji cidden nanay
9836. satır: olursa olsun güçlü bir psikoloji aşarsınız
6001. satır: benim psikoloji artık geç kurtaramayız

Seçilen Cümle (10605. satır): pashanimden adelee atlıyorum psikoloji derece gidik
En Benzer 3 Cümle:
2682. satır: psikoloji gidik
3690. satır: sizde psikoloji gidik
3594. satır: benim psikoloji ondan gidik galiba

Seçilen Cümle (10000. satır): hoccaofficial gün dedik keşke yazmasaydın gene dayanamadın tabi huy değişir miseni böyle seviyoruzyükseliş olunca kimse dayanamayacak satacak yine kimse dayanamayıp yukarıdan alacak iş söylem değil psikoloji yönetili

In [7]:


# Excel dosyasını oku
excel_file_path = 'final_data_file.xlsx'  # Dosya yolu ve adını güncelleyin
df = pd.read_excel(excel_file_path)

# Kullanıcı tarafından seçilen 5 cümleyi belirle
selected_indices = [7865, 5678, 4444, 156, 9]  # Seçmek istediğiniz satır numaralarını güncelleyin
selected_sentences = df['sentence'].iloc[selected_indices].tolist()

# TfidfVectorizer kullanarak metin vektörlerini oluştur
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['sentence'])

# Her seçilen cümle için benzerlik oranlarını ve satır numaralarını hesapla
similarities = []
for sentence_index, sentence in zip(selected_indices, selected_sentences):
    sentence_vector = vectorizer.transform([sentence])
    cosine_similarities = linear_kernel(sentence_vector, tfidf_matrix).flatten()
    most_similar_indices = cosine_similarities.argsort()[:-5:-1]  # En benzer 3 cümleyi seç
    similarity_scores = cosine_similarities[most_similar_indices]
    
    similar_sentences_info = []
    for i, similar_index in enumerate(most_similar_indices):
        similar_sentence = df['sentence'].iloc[similar_index]
        similar_sentences_info.append({
            'Sentence': similar_sentence,
            'Similarity Score': similarity_scores[i],
            'Row Number': similar_index,
        })
    
    similarities.append({
        'Selected Sentence': sentence,
        'Similar Sentences': similar_sentences_info,
    })

# Sonuçları yazdır
for index, similarity in enumerate(similarities):
    print(f"\nSeçilen Cümle {index + 1} (Satır Numarası {selected_indices[index]}): {similarity['Selected Sentence']}")
    for i, similar_info in enumerate(similarity['Similar Sentences']):
        print(f"\tBenzer Cümle {i + 1} (Satır Numarası {similar_info['Row Number']}): {similar_info['Sentence']} (Benzerlik Oranı: {similar_info['Similarity Score']:.4f})")


Seçilen Cümle 1 (Satır Numarası 7865): psikoloji okumalıydım hata ettim aklıma geliyor
	Benzer Cümle 1 (Satır Numarası 7865): psikoloji okumalıydım hata ettim aklıma geliyor (Benzerlik Oranı: 1.0000)
	Benzer Cümle 2 (Satır Numarası 5151): psikoloji okumalıydım (Benzerlik Oranı: 0.5522)
	Benzer Cümle 3 (Satır Numarası 380): psikoloji tercih ettim (Benzerlik Oranı: 0.2764)
	Benzer Cümle 4 (Satır Numarası 475): psikoloji kitabı okuyorum konuda aklıma bir yazar şarkı sözü yazarı geliyor (Benzerlik Oranı: 0.2645)

Seçilen Cümle 2 (Satır Numarası 5678): psikolojik duygusal faktör işin içinde dediğin
	Benzer Cümle 1 (Satır Numarası 5678): psikolojik duygusal faktör işin içinde dediğin (Benzerlik Oranı: 1.0000)
	Benzer Cümle 2 (Satır Numarası 1300): işin ı psikoloji (Benzerlik Oranı: 0.3759)
	Benzer Cümle 3 (Satır Numarası 4555): psikoloji işin i (Benzerlik Oranı: 0.3759)
	Benzer Cümle 4 (Satır Numarası 6029): taraftar psikoloji işin içinde (Benzerlik Oranı: 0.3621)

Seçilen Cümle 3 (Satır Nu